In [16]:
from ModelSchema.NeuralNetWork import dataset, train_and_test, model

import os
from torch.utils.data import DataLoader
import torch
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
from torchvision import models
from torchinfo import summary


test = dataset.SpectraDataset('../database/spectras/test')
loader_test = DataLoader(test, batch_size= 32, pin_memory= True, shuffle=True)


In [4]:
mobilenet = models.get_model('mobilenet_v3_large',weights = 'MobileNet_V3_Large_Weights.IMAGENET1K_V2')

shufflenet = models.get_model('shufflenet_v2_x2_0',weights = 'ShuffleNet_V2_X2_0_Weights.IMAGENET1K_V1')

resnet = models.get_model('resnet152',weights = 'ResNet152_Weights.IMAGENET1K_V2')

In [64]:
shufflenet.fc.out_features

2

In [27]:
for name, param in resnet.named_parameters():
        if 'fc' not in name:
            param.requires_grad = False
        else:
              print(name)
              param.requires_grad = True

fc.weight
fc.bias


In [28]:
for name, param in resnet.named_parameters():

    if param.requires_grad:
        print(f"Nome: {name} | Requer gradiente: {param.requires_grad}")

Nome: fc.weight | Requer gradiente: True
Nome: fc.bias | Requer gradiente: True


In [29]:
resnet.fc

Linear(in_features=2048, out_features=1000, bias=True)

In [30]:
in_features = resnet.fc.in_features
resnet.fc = torch.nn.Linear(in_features, 2)

In [31]:
torch.save(resnet,'../models/base_models/'+'resnet')

In [17]:
model = torch.load('../models/trained_models/resnet_200.pth', weights_only=False)  

In [18]:
for name, param in model.named_parameters():

    if param.requires_grad:
        print(f"Nome: {name} | Requer gradiente: {param.requires_grad}")

model

Nome: adaptador.weight | Requer gradiente: True
Nome: adaptador.bias | Requer gradiente: True
Nome: base_model.fc.weight | Requer gradiente: True
Nome: base_model.fc.bias | Requer gradiente: True


ConvolutionalNet(
  (adaptador): Conv2d(1, 3, kernel_size=(2, 2), stride=(1, 1))
  (base_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    

In [19]:
pressao_total = 0


pressao = []
temperatura = []
for X,y in loader_test:
    model.eval()
    with torch.no_grad():
        pred = model(X)

        temperatura_pred = pred[: ,0]*(373.15-273.15) + 273.15
        temperatura_real = y[:,0] *(373.15-273.15) + 273.15

        pressao_pred = pred[:, 1]* (1- 0.01) + 0.01
        pressao_real = y[:, 1] * (1- 0.01) + 0.01
        


print(np.mean(pressao), np.mean(temperatura))

nan nan


In [49]:
X , env = test[int(np.random.uniform(0,200))]
print(X)
model.eval()
with torch.no_grad():
    pred = model(X.unsqueeze(0))

tensor([[[-223.6672, -226.3535, -228.1905,  ..., -229.2529, -227.7433,
          -224.9822],
         [-223.3766, -225.8926, -226.6548,  ..., -226.0008, -226.8926,
          -225.4104],
         [-222.7062, -224.3936, -227.4317,  ..., -227.4777, -226.7649,
          -224.4631],
         ...,
         [-222.8204, -224.0515, -224.7342,  ..., -225.7582, -224.3983,
          -224.2156],
         [-223.3112, -223.2260, -224.1795,  ..., -226.0372, -223.3821,
          -222.7327],
         [-223.1113, -225.1295, -226.6313,  ..., -227.2908, -226.4879,
          -224.2696]]])


In [50]:
env[0] *(373.15-273.15) + 273.15 , pred[0][0]*(373.15-273.15) + 273.15

(tensor(281.1433), tensor(291.0499))

In [51]:
env[1] * (1- 0.01) + 0.01 , pred[0][1]* (1- 0.01) + 0.01

(tensor(0.4122), tensor(0.3949))